# MATH 210 Introduction to Mathematical Computing

## Project I: Solving First Order Linear PDEs using `sympy.solvers.pde`

SymPy is one of the core mathematical packages within Python and the subpackage `sympy.solvers.pde` addresses how to separate, classify and solve 3 different types of partial differential equations (PDEs) (see the [documentation](http://docs.sympy.org/latest/modules/solvers/pde.html)):

1. First order linear homogeneous PDEs with constant coefficients: 
$$
a \frac{d}{dx} f(x,y) + b \frac{d}{dy} f(x,y) + cf(x,y) = 0
$$
where $a,b$ and $c$ are constants.

2. First order linear PDEs with constant coefficients:
$$
a \frac{d}{dx} f(x,y) + b \frac{d}{dy} f(x,y) + cf(x,y) = G(x,y)
$$
where $a,b$ and $c$ are constants and $G(x,y)$ can be an arbitrary function in $x$ and $y$.

3. First order linear PDEs with variable coefficients:
$$
a(x,y) \frac{d}{dx} f(x,y) + b(x,y) \frac{d}{dy} f(x,y) + c(x,y)f(x,y) = G(x,y)
$$
where $a(x,y), b(x,y), c(x,y)$ and $G(x,y)$ are arbitrary functions in $x$ and $y$.

Our goal in this notebook is to explore three functions in the subpackage `sympy.solvers.pde` which solve **partial differential equations**.

By the end of the notebook, the reader will be able to classify and separate partial differential equations, as well as implement the following functions to solve in order to solve first order linear partial differential equations:
* `sympy.solvers.pde.pde_1st_linear_constant_coeff_homogeneous`
* `sympy.solvers.pde.pde_1st_linear_constant_coeff`
* `sympy.solvers.pde.pde_1st_linear_variable_coeff`



## Contents

1. Notation
2. Separating PDEs: `sympy.solvers.pde.pde_separate`
3. Solving PDEs: introduction to `pdsolve`
4. Classifying PDEs: `sympy.solvers.pde.classify_pde`
5. Solving PDEs: implementing various functions
    * Examples
    * `checkpdesol`
    * `sympy.solvers.pde.pde_1st_linear_constant_coeff_homogeneous`
    * `sympy.solvers.pde.pde_1st_linear_constant_coeff`
    * `sympy.solvers.pde.pde_1st_linear_variable_coeff`
        * `dsolve`

## 1. Notation

Throughout this notebook, I will denote $$ \frac{d}{dx} u(x,t) \ \ \text{and} \ \ \frac{d}{dt} u(x,t) $$ ie. the partial derivatives of $u$ with respect to $x$ and $t$, as $u_x$ and $u_t$ respectively.

Additionally, the second partial derivatives of $u$ with repsect to $x$ and $t$ respectively will be given by: $u_{xx} \ \ \text{and} \ \ u_{tt}$.

In cases where the equations are in terms of $f(x,y)$, we assume that $u = f(x,y)$.

When defining our equation, we rearrange it so that all the terms are on one side of the equation and other side equals 0.

## 2. Separating PDEs: `sympy.solvers.pde.pde_separate`

Separating PDEs can be done in two ways: using **addition** or **multiplication**. Both simply try to separate the differential equation so that one variable lies on one side of the equation and the other variable on the other, provided the PDE only depends on two variables.

`pde_separate` takes 4 input parameters:
* **`eq`** (partial differential equation)
* **`fun`** (original function $F(x,y,z)$)
* **`sep`** (list of separated functions $[u(x,y),Z(z)]$)
* **`strategy`** (separation stragegy - either 'add' or 'mul' depending on the chosen approach)

and returns the equation fully separated according to whether it was separated using the addition or multiplication approach.

#### Example 2.1

We will separate the equation

$$
3u_x = 8e^u u_t
$$

using the **addition** approach:

$$
u(x,t) = X(x) + T(t)
$$

In [25]:
from sympy import E, Eq, Function, pde_separate, Derivative as D
from sympy.abc import x, t
u, X, T = map(Function, 'uXT')

In [26]:
eq = Eq(3*D(u(x, t), x), 8*E**(u(x, t))*D(u(x, t), t))
pde_separate(eq, u(x, t), [X(x), T(t)], strategy='add')

[exp(-X(x))*Derivative(X(x), x), 8*exp(T(t))*Derivative(T(t), t)/3]

We have obtained the separation as follows:
$$
u_x = X'(x) \ \text{and} \ u_t = T'(t)
$$


$$
3X'(x) = 8e^{X(x) + T(t)}T'(t)
$$


$$
3X'(x) = 8e^{X(x)}e^{T(t)}T'(t)
$$


$$
e^{-X(x)}X'(x) = \frac{8}{3}e^{T(t)}T'(t)
$$

as given by our output of `[exp(-X(x))*Derivative(X(x), x), 8*exp(T(t))*Derivative(T(t), t)/3]`, which displays either side of the equation separated by a comma as a list.

#### Example 2.2

We will separate the equation $$u_{xx} + u = 5u_{t}$$

using the **multiplication** approach:
$$u(x,t) = X(x)T(t)$$

In [27]:
eq = Eq(D(u(x, t), x, 2) + u(x,t), D(5*u(x, t), t))
pde_separate(eq, u(x, t), [X(x), T(t)], strategy='mul')

[1 + Derivative(X(x), x, x)/X(x), 5*Derivative(T(t), t)/T(t)]

We have obtained the separation as follows:

$$
u_{xx} = X''(x)T(t) \ \text{and} \ u_{t} = X(x)T'(t)
$$


$$
X''(x)T(t) + X(x)T(t) = 5X(x)T'(t)
$$


$$
\frac{X''(x) + X(x)}{X(x)} = \frac{5T'(t)}{T(t)}
$$

$$
1 + \frac{X''(x)}{X(x)} = \frac{5T'(t)}{T(t)}
$$

as given by our output of `[1 + Derivative(X(x), x, x)/X(x), 5*Derivative(T(t), t)/T(t)]`, which again displays either side of the equation as a list.

Equivalently, for each of our examples above, there exists the SymPy functions `pde_separate_add` and `pde_separate_mul` which return the same outputs as above:

In [28]:
from sympy import pde_separate_add
from sympy import pde_separate_mul

In [29]:
eq = Eq(3*D(u(x, t), x), 8*E**(u(x, t))*D(u(x, t), t))
pde_separate_add(eq, u(x, t), [X(x), T(t)])

[exp(-X(x))*Derivative(X(x), x), 8*exp(T(t))*Derivative(T(t), t)/3]

In [30]:
eq = Eq(D(u(x, t), x, 2) + u(x,t), D(5*u(x, t), t))
pde_separate_mul(eq, u(x, t), [X(x), T(t)])

[1 + Derivative(X(x), x, x)/X(x), 5*Derivative(T(t), t)/T(t)]

We can time these different methods of separating PDEs to see which processes the fastest... let's go!

In [27]:
%timeit pde_separate_add

10000000 loops, best of 3: 24 ns per loop


In [28]:
%timeit pde_separate_mul

10000000 loops, best of 3: 21.6 ns per loop


Having tested them both multiple times, `pde_separate_mul` appears to be faster in computing the separation of these 2 PDEs.

## 3. Solving PDEs: introduction to `pdsolve`

`pdsolve` allows us to solve all kinds of partial differential equations. It can take 4 input parameters:
* `eq` is the supported PDE of any type.
* `f(x,y)` is a function of 2 variables $x$ and $y$ that gives us our PDE with its derivatives. In many cases, the function is automatically detected and therefore it is an optional parameter.
* `hint` is simply the method we want to use to solve our PDE. For example, if we use `classify_pde(eq,f(x,y))`, we will have all the possible hints that are available for our particular partial differential equation. The default hint is just 'default' and this will use whichever is returned first by classify_pde().
* `solvefun` is set by default to be F if we do not choose the value. This is the convention used for arbitrary functions returned by the solver.

There are 3 possible options for 'hint':
1. As explained above, 'default' returns the first hint returned by classify_pde().
2. Setting `hint` equal to 'all' returns a dictionary of possible solution terms. However, if a hint causes `pdsolve` to print 'NotImplementedError', then the value of this hint's key is the exception object raised. The dictionary may also include some special keys:
    (a) 'order' gives us the order of the PDE
    (b) 'default' gives us the solution that would have been returned had our original hint been left at 'default'.
3. 'all_integral' is the same as 'all' apart from the fact that it will only return the 'all_integral' hint. This is particularly helpful if the integral required to solve the PDE is difficult or impossible and just using 'all' would cause `pdsolve` to stop.


#### Example 3.1

Let's solve the PDE
$$
4 + \frac{5}{3} \frac{u_x}{u} + 7 \frac{u_y}{u} = 0
$$

In [31]:
from sympy.solvers.pde import pdsolve
from sympy import Function, diff, Eq
from sympy.abc import x, y

In [32]:
f = Function('f')
u = f(x, y)
ux = u.diff(x)
uy = u.diff(y)
eq = Eq(4 + (5/3)*(ux/u) + 7*(uy)/u)
pdsolve(eq)

Eq(f(x, y), F(7*x - 1.66666666666667*y)*exp(-0.128755364806867*x - 0.540772532188841*y))

We have successfully implemented `pdsolve` and obtained the answer
$$
f(x,y) = F(7x - \frac{5}{3} y)e^{\frac{-30}{233}x - \frac{126}{233}y}
$$

where $F$ is an arbitrary function of $7x - \frac{5}{3}y$.

## 4. Classifying PDEs: `sympy.solvers.pde.classify_pde`

There are 3 different types of PDEs that this function can recognise:
* 1st order linear homoegenous PDEs with constant coefficients
* 1st order linear PDEs with constant coefficients
* 1st order linear PDEs with variable coefficients

The `classify_pde` function returns a tuple of possible classifications for a given PDE.

In [33]:
from sympy.solvers.pde import classify_pde

#### Example 4.1

Let's classify the following PDE:

$$
4u_x + 5 = 3u_t
$$

In [34]:
f = Function('f')
u = f(x, y)
ux = u.diff(x)
ut = u.diff(t)
eq = Eq(4*ux + 5 - 3*ut)
classify_pde(eq)

('1st_linear_constant_coeff', '1st_linear_constant_coeff_Integral')

This means that the most likely classification for this PDE is '1st order linear PDE with constant coefficients' and this will be the deafult classification.

## 5. Solving PDEs: implementing various functions

#### Example 5.1

Using `pdsolve`, we can only solve first order partial differential equations.

We will prove this by attempting to solve the first order heat equation

$$
u_x = u_{tt}
$$

and see that this builtin function cannot solve this equation.

In [35]:
from sympy import E, Eq, Function, pdsolve, Derivative as D
from sympy.abc import x, t

In [36]:
f = Function('f')
pdsolve(f(x,t).diff(x) - (f(x,t).diff(t)).diff(t))

NotImplementedError: psolve: Cannot solve Derivative(f(x, t), x) - Derivative(f(x, t), t, t)

As expected, we cannot solve the first order heat equation.

#### Example 5.2

We will now try and solve a simple first order PDE:

$$
2u_x + u_y = 2y
$$

In [37]:
f=Function('f')
u=f(x,y)
ux = u.diff(x)
uy = u.diff(y)
pdsolve(2*ux + uy - 2*y)

Eq(f(x, y), -4*x**2/25 + 16*x*y/25 + 9*y**2/25 + F(x - 2*y))

We have obtained the solution $u(x,y) = -\frac{4}{25}x^2 + \frac{16}{25}xy + \frac{9}{25}y^2 + F(x-2y)$.

In order to check our solution, we differentiate $u$ as follows:
$$
u_x = -\frac{8}{25}x + \frac{16}{25}y + F(x-2y) \ \ \text{and} \ \ u_y = \frac{16}{25}x + \frac{18}{25}y - 2F(x-2y)
$$

$$
2u_x + u_y = -\frac{16}{25}x + \frac{16}{25}x + \frac{32}{25}y + \frac{18}{25}y + 2F(x-2y) - 2F(x-2y) = \frac{50}{25}y = 2y
$$

Therefore our equation is satisfied and $u(x,y) = -\frac{4}{25}x^2 + \frac{16}{25}xy + \frac{9}{25}y^2 + F(x-2y)$ is indeed the solution.

### `checkpdesol`

Another method of checking our answer satisfies the original equation is the built in function `checkpdesol`.

Using the same example as above we demonstrate the use of `checkpdesol`:

In [38]:
from sympy.solvers.pde import checkpdesol

In [39]:
f = Function('f')
u = f(x,y)
ux = u.diff(x)
uy = u.diff(y)
eq = 2*ux + uy - 2*y
sol = pdsolve(eq)
assert checkpdesol(eq, sol)
checkpdesol(eq,sol)

(True, 0)

The returned tuple gives us that the answer is indeed a solution, given by 'True'.
If instead `checkpdesol` returned 'False', instead of 0 as the second element in the tuple, we would see the expression obtained when substituting our answer into the given equation. For example, if we add an extra equation and check if our previous answer is also a solution for this equation:

In [40]:
f = Function('f')
u = f(x,y)
ux = u.diff(x)
uy = u.diff(y)
eq = 2*ux + uy - 2*y
sol = pdsolve(eq)
assert checkpdesol(eq, sol)
eq = ux - 4*uy + x
checkpdesol(eq, sol)

(False,
 -47*x/25 - 56*y/25 + 9*Subs(Derivative(F(_xi_1), _xi_1), (_xi_1,), (x - 2*y,)))

We obtain that our solution does not satisfy the equation $u_x - 4u_y + x$ and instead when substituted in, returns
$$
-\frac{47}{25}x - \frac{56}{25} y + 9F(x-2y)
$$

as given by `-47*x/25 - 56*y/25 + 9*Subs(Derivative(F(_xi_1), _xi_1), (_xi_1,), (x - 2*y,))`.

### `sympy.solvers.pde.pde_1st_linear_constant_coeff_homogeneous`

Using `classify_pde` tells us what type of PDE we are trying to solve, which makes solving it much easier, because we can then implement a specific function which makes solving it even easier.

First order linear homogeneous PDEs with constant coefficients have the following form: 
$$
a \frac{d}{dx} f(x,y) + b \frac{d}{dy} f(x,y) + cf(x,y) = 0
$$
where $a,b$ and $c$ are constants.

The general solution is of the form:

$$
e^{\frac{-c(ax+by)}{a^2 + b^2}}F(bx-ay)
$$ 

where $F$ is an arbitrary function of $bx-ay$, as shown below:

In [41]:
from sympy.abc import a,b,c
from sympy import pprint

In [42]:
f = Function('f')
u = f(x,y)
ux = u.diff(x)
uy = u.diff(y)
genform = a*ux + b*uy + c*u
pprint(genform)

  ∂               ∂                      
a⋅──(f(x, y)) + b⋅──(f(x, y)) + c⋅f(x, y)
  ∂x              ∂y                     


In [43]:
pprint(pdsolve(genform))

                         -c⋅(a⋅x + b⋅y) 
                         ───────────────
                              2    2    
                             a  + b     
f(x, y) = F(-a⋅y + b⋅x)⋅ℯ               


#### Example 5.3

Let's solve $27u_x + 13u_y + 4u = 0$.

From the general form, we expect the answer to equal $$u(x,y) = e^{\frac{-4(27x+13y)}{27^2 + 13^2}}F(13x-27y) = e^{\frac{-54x - 26y}{449}}F(13x-27y)$$

In [44]:
f = Function('f')
u = f(x,y)
ux = u.diff(x)
uy = u.diff(y)
eq = 27*ux + 13*uy + 4*u
classify_pde(eq)

('1st_linear_constant_coeff_homogeneous',)

From this we know that this equation is indeed a first order linear homogenous equation with constant coefficients. We can now solve the equation:

In [46]:
f = Function('f')
u = f(x,y)
ux = u.diff(x)
uy = u.diff(y)
eq = 27*ux + 13*uy + 4*u
pdsolve(eq)

Eq(f(x, y), F(13*x - 27*y)*exp(-54*x/449 - 26*y/449))

In [47]:
pprint(pdsolve(eq))

                            54⋅x   26⋅y
                          - ──── - ────
                            449    449 
f(x, y) = F(13⋅x - 27⋅y)⋅ℯ             


Which is the answer we expected from the general form.

### `sympy.solvers.pde.pde_1st_linear_constant_coeff`

First order linear PDEs with constant coefficients have the following form:
$$
a \frac{d}{dx} f(x,y) + b \frac{d}{dy} f(x,y) + cf(x,y) = G(x,y)
$$
where $a,b$ and $c$ are constants and $G(x,y)$ can be an arbitrary function in $x$ and $y$.


In this case, it is enough to find the general solution of $$a \frac{d}{dx} f(x,y) + b \frac{d}{dy} f(x,y) + cf(x,y) = 0$$ and a particular solution to our original equation and sum the two together.

As before, we have that the general solution to the homogeneous equation is as follows:
$$
e^{\frac{-c(ax+by)}{a^2 + b^2}}F(bx-ay)
$$

In order to find a particular solution to the inhomogeneous equation, we use the method of characteristics by changing the variables in the equation.

Our change of coordinates is given by:
$$
\xi = ax + by
$$
$$
\eta = bx - ay
$$

This gives us that:
$$
u_x = u_{\xi} \frac{\partial \xi}{\partial x} + u_{\eta} \frac{\partial \eta}{\partial x} = au_{\xi} + bu_{\eta}
$$
$$
u_y = u_{\xi} \frac{\partial \xi}{\partial y} + u_{\eta} \frac{\partial \eta}{\partial y} = bu_{\xi} - au_{\eta}
$$

And so:

$$
au_x + bu_y = a(au_{\xi} + bu_{\eta}) + b(bu_{\xi} - au_{\eta}) = a^2u_{\xi} + b^2u_{\xi} + abu_{\eta} - abu_{\eta} = (a^2 + b^2)u_{\xi}
$$

Thus our inhomogeneous equation becomes:

$$
(a^2 + b^2)u_{\xi} + cu = G(\xi,\eta)
$$



ie. 
$$
u_{\xi} = \frac{G(\xi,\eta)}{a^2 + b^2} - \frac{c}{a^2+b^2} u
$$

Using variation of parameters we obtain that the particular solution is as follows:

$$
e^{-\frac{c}{a^2+b^2}\xi} \int \frac{G(\xi,\eta)}{a^2 +  b^2} e^{\frac{c}{a^2+b^2}\xi} d\xi
$$

and therefore the general solution to our inhomogeneous equation is given by:

$$
u(\xi,\eta) = e^{-\frac{c}{a^2+b^2}\xi} \left ( F(\eta) + \int \frac{G(\xi,\eta)}{a^2 +  b^2} e^{\frac{c}{a^2+b^2}\xi} d\xi \right )
$$

Once we have obtained this solution we can substitute our change of variables back in to find the solution in terms of $x$ and $y$.

#### Example 5.4

Let's solve the equation $5u_x - 6u_y + 3u = e^{2x +y}$.

Using our general form, we have that $a=5, b=-6, c=3, G(x,y) = e^{2x+y}, \xi = 5x - 6y$ and $\eta = -6x - 5y$.

$2x + y = \frac{4\xi - 17\eta}{61}$

Therefore we would expect our final solution to be given by
$$
u(\xi,\eta) = e^{-\frac{3}{61}\xi} \left ( F(\eta) + \int \frac{e^{\frac{4 \xi - 17 \eta}{61}}}{61} e^{\frac{3}{61}\xi} d\xi \right )
$$

which is simplified to
$$
u(\xi,\eta) = e^{-\frac{3}{61} \xi} \left ( F(\eta) + \frac{e^{\frac{7\xi - 17\eta}{61}}}{7} \right )
$$

and then
$$
u(x,y) = e^{\frac{-15x + 18y}{61}} \left ( F(-6x-5y) + \frac{e^\frac{137x + 43y}{61}}{7} \right )
$$

Now let's run our function and check if this is indeed what we obtain. First we classify the PDE:

In [59]:
f = Function('f')
u = f(x,y)
ux = u.diff(x)
uy = u.diff(y)
eq = 5*ux - 6*uy + 3*u - E**(2*x + y)
classify_pde(eq)

('1st_linear_constant_coeff', '1st_linear_constant_coeff_Integral')

This tells us that the most likely type of equation we have is indeed a first order linear equation with constant coefficients. Now we can solve our equation:

In [60]:
pdsolve(eq)

Eq(f(x, y), (F(-6*x - 5*y) + exp(65*x/61 - 78*y/61)**(7/13)*exp(102*x/61 + 85*y/61)/7)*exp(-15*x/61 + 18*y/61))

In [61]:
pprint(pdsolve(eq))

          ⎛                              7/13              ⎞               
          ⎜                ⎛ 65⋅x   78⋅y⎞      102⋅x   85⋅y⎟               
          ⎜                ⎜ ──── - ────⎟      ───── + ────⎟    15⋅x   18⋅y
          ⎜                ⎜  61     61 ⎟        61     61 ⎟  - ──── + ────
          ⎜                ⎝ℯ           ⎠    ⋅ℯ            ⎟     61     61 
f(x, y) = ⎜F(-6⋅x - 5⋅y) + ────────────────────────────────⎟⋅ℯ             
          ⎝                               7                ⎠               


We have that $\frac{65}{61}$ multiplied by $\frac{7}{13}$ and then added to $\frac{102}{61}$ gives us $\frac{137}{61}$, as expected for the power of $e$ with respect to $x$, and the same process with give us $\frac{43}{61}$ for the coefficient of $y$.

### `sympy.solvers.pde.pde_1st_linear_variable_coeff`

First order linear PDEs with variable coefficients are of the form:
$$
a(x,y) \frac{d}{dx} f(x,y) + b(x,y) \frac{d}{dy} f(x,y) + c(x,y)f(x,y) = G(x,y)
$$
where $a(x,y), b(x,y), c(x,y)$ and $G(x,y)$ are arbitrary functions in $x$ and $y$.

In order to solve this equation, again we use a change of variables with $\xi = x$ and $\eta$ is equal to the constant in the solution to the ordinary differential equation $\frac{dy}{dx} = \frac{b}{a}$.

By using these substitutions, we reduce our PDE to a linear ODE of the form:
$$
a(\xi,\eta)\frac{du}{d\xi} + c(\xi,\eta)u - G(\xi,\eta) = 0
$$

and this can be solved using `dsolve` (the built in function for solving ordinary differential equations).

#### Example 5.5

Let's solve the PDE $xu_x + yu_y + xyu = 2x$.

We have the substitutions $\xi = x$ and

$$
\frac{dy}{dx} = \frac{y}{x}
$$

which leads to $$ \frac{dy}{y} = \frac{dx}{x} $$

and therefore $$ \eta = \ln  \left ( \frac{y}{x} \right ) $$

This gives us $$\xi u_{\xi} + \xi^2 e^\eta u - 2\xi = 0$$ which we can solve using `dsolve`.

### `dsolve`

Before we use `dsolve`, let us confirm that our equation is of the right kind:

In [28]:
f = Function('f')
u = f(x,y)
ux = u.diff(x)
uy = u.diff(y)
eq = x*ux + y*uy + x*y*u - 2*x
classify_pde(eq)

('1st_linear_variable_coeff',)

Indeed, our equation is a first order linear partial differential equation with variable coefficients. Now from our calculations above, let's check what `dsolve` comes up with.

`dsolve` essentially works in the same way as `pdsolve`, except that `dsolve` can only solve differential equations of one variable, in our case, $\xi$.

In [32]:
import numpy as np
from sympy import dsolve

In [34]:
f = Function('f')
xi = x
eta=y
u = f(xi)
uxi = u.diff(xi)
eq = xi*uxi - xi**2*E**eta*u - 2*xi
dsolve(eq)

Eq(f(x), (C1 + sqrt(2)*sqrt(pi)*erf(sqrt(2)*x*sqrt(exp(y))/2)/sqrt(exp(y)))*exp(x**2*exp(y)/2))

Which will give us the same solution when substituting our $x$ and $y$ back in as below:

In [35]:
f = Function('f')
u = f(x,y)
ux = u.diff(x)
uy = u.diff(y)
eq = x*ux + y*uy + x*y*u - 2*x
pdsolve(eq)

Eq(f(x, y), Piecewise(((2*x + F(y/x))*exp(-x*y/2), Eq(y/x, 0)), ((F(y/x) - sqrt(2)*I*sqrt(pi)*erf(sqrt(2)*I*x*sqrt(y/x)/2)/sqrt(y/x))*exp(-x*y/2), True)))

Which is expressed as
$$
f(x,y) = \left \{
         \begin{array}{ll}
            \left ( 2x + F \left ( \frac{y}{x} \right ) \right )e^{-\frac{xy}{2}} & \quad \text{for} \ \frac{y}{x} = 0 \\
            \left ( F \left ( \frac{y}{x} \right ) - \frac{\sqrt{2} i}{\sqrt{\frac{y}{x}}} \sqrt{\pi} \ \text{erf} \left ( \frac{ix}{2} \sqrt{2} \sqrt{\frac{y}{x}} \right ) \right ) e^{-\frac{xy}{2}} & \quad \text{otherwise}
        \end{array}
    \right.
$$

where 'erf' is the error function of a complex argument.

There we have it! We have successfully learned how to solve first order linear partial differential equations by recognising the type of PDE they are, as well as separate and classify them in order to solve them by hand and check our solutions are correct.

### References

* [University of California, Santa Barbara](http://web.math.ucsb.edu/~grigoryan/124A.pdf)
* [SymPy Documentation](http://docs.sympy.org/latest/modules/solvers/pde.html)
* [SymPy Documentation docstring](https://searchcode.com/codesearch/view/78124480/)